In [ ]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from enum import Enum
import cv2
from tensorflow.keras import backend as K # Importing Keras backend (by default it is Tensorflow)
import gc
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adam
from keras_adabound import AdaBound
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Input, Conv2D,BatchNormalization
from tensorflow.keras.layers import Activation,SpatialDropout2D,AvgPool2D
from tensorflow.keras.layers import MaxPool2D,Dropout,GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from sklearn.metrics import cohen_kappa_score, accuracy_score
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import DenseNet121,DenseNet169,DenseNet201
from tensorflow.keras.models import load_model
from numpy import dstack
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
import seaborn as sns

In [ ]:
# define seed number to have reproducible experiments.
seed = 3352024
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
def load_path(path):
    '''
    load MURA dataset
    '''
    dataset = [] 
    for body in os.listdir(path):
        body_part = body
        path_p = path+'/'+str(body)
        for id_p in os.listdir(path_p):
            patient_id = id_p
            path_id = path_p+'/'+str(id_p)
            for lab in os.listdir(path_id):
                if lab.split('_')[-1]=='positive': 
                    label = 1 
                elif lab.split('_')[-1]=='negative':
                    label= 0
                path_l = path_id+'/'+str(lab)
                for img in os.listdir(path_l):  
                    img_path = path_l + '/' + str(img)
                    dataset.append(
                        {
                            'body_part': body_part,
                            'patient_id': patient_id,
                            'label': label,
                            'img_path': img_path
                        }
                    )
    return dataset

In [ ]:
path = 'MURA-v1.1/train'
os.listdir(path)

In [ ]:
dataset_train= load_path(path)
df_train = pd.DataFrame(dataset_train)
df_train.head()

In [ ]:
dataset_test = load_path(path = 'MURA-v1.1/valid')
df_test = pd.DataFrame(dataset_test)
df_test.head()

In [ ]:
dataset_all=pd.concat([df_train,df_test])
print(dataset_all.body_part.value_counts())
print(dataset_all.label.value_counts())

In [ ]:
print(df_train.shape[0]/dataset_all.shape[0])
print(df_test.shape[0]/dataset_all.shape[0])

In [ ]:
df_train[df_train.body_part=='XR_WRIST'].shape[0]

In [ ]:
def cnn_builder(
        input_shape=(256, 256, 3),
        starting_filters=32,
        conv_layers=1,
        conv_strides=(1, 1),
        conv_kernel=(3, 3),
        convs_per_layer=1,
        batch_norm=False,
        pooling="max",
        dropout=None,
        pool_size=(2, 2),
        pool_strides=(2, 2),
        last_pooling=None,
        spatial_dropout=None,
        last_dropout=None,
        numdense=1
):
    inputs = Input(
        shape=input_shape,
        name="input"
    )
    x = inputs
    for conv_level in range(conv_layers):
        current_filters = starting_filters * (2 ** conv_level)
        for conv_number in range(convs_per_layer):
            x = Conv2D(
                filters=current_filters,
                kernel_size=conv_kernel,
                strides=conv_strides,
                name=f"conv_{conv_level}_{conv_number}",
                padding='same'
            )(x)
            if batch_norm:
                x = BatchNormalization(name=f"bn_{conv_level}_{conv_number}")(x)
            x = Activation("relu", name=f"conv_{conv_level}_{conv_number}_relu")(x)
        if spatial_dropout:
            x = SpatialDropout2D(spatial_dropout, name=f"sp_dropout_{conv_level}")(x)
        if pooling == 'avg':
            x = AvgPool2D(pool_size=pool_size, 
                          strides=pool_strides,
                          name=f"mp_{conv_level}",
                          padding='same')(x)
        elif pooling == 'max':
            x = MaxPool2D(pool_size=pool_size,
                          strides=pool_strides,
                          name=f"mp_{conv_level}",
                          padding='same')(x)
        if dropout:
            x = Dropout(dropout, name=f"dropout_{conv_level}")(x)
    if last_pooling == "avg":
        x = GlobalAveragePooling2D(name=f"lp_{last_pooling}")(x)
    elif last_pooling == "max":
        x = GlobalMaxPooling2D(name=f"lp_{last_pooling}")(x)
    x = Flatten(name="flatten")(x)
    if(numdense==2):
        x=Dense(10,activation='relu',name='dense1')(x)
    if last_dropout:
        x = Dropout(last_dropout, name="last_dp")(x)
    output = Dense(1, activation='sigmoid', name="output")(x)
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
def metrics():
    return [
        AUC(name="auc"),
        BinaryAccuracy("accuracy"),
        CohenKappa(name="kappa", num_classes=2)
    ]

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)

In [ ]:
cnn_model41avg = cnn_builder(starting_filters=32,
                        conv_layers=4,
                        convs_per_layer=1,
                        pooling='avg',
                        batch_norm=True,
                        dropout=0.2,
                        pool_strides=(2, 2),
                        numdense=1)
cnn_model41avg.summary()

In [ ]:
cnn_model41max = cnn_builder(starting_filters=32,
                        conv_layers=4,
                        convs_per_layer=1,
                        pooling='max',
                        batch_norm=True,
                        dropout=0.2,
                        pool_strides=(2, 2),
                        numdense=1)

cnn_model41max.summary()

In [ ]:
cnn_model51avg = cnn_builder(starting_filters=32,
                        conv_layers=5,
                        convs_per_layer=1,
                        pooling='avg',
                        batch_norm=True,
                        dropout=0.2,
                        pool_strides=(2, 2),
                        numdense=1)

cnn_model51avg.summary()

In [ ]:
cnn_model51max = cnn_builder(starting_filters=32,
                        conv_layers=5,
                        convs_per_layer=1,
                        pooling='max',
                        batch_norm=True,
                        dropout=0.2,
                        pool_strides=(2, 2),
                        numdense=1)

cnn_model51max.summary()

In [ ]:
print('In the train dataset')
for bp in df_train.body_part.unique():
    sp0=df_train[(df_train.body_part==bp) & (df_train.label==0)].shape[0]
    sp1=df_train[(df_train.body_part==bp) & (df_train.label==1)].shape[0]
    print(f'{bp} 0: {sp0} ({100*sp0/(sp0+sp1): .2f}%)')
    print(f'{bp} 1: {sp1} ({100*sp1/(sp0+sp1): .2f}%)')
    print(f'{bp} all: {sp0+sp1} ({100*(sp0+sp1)/(df_train.shape[0]):.2f}%)')

In [ ]:
print('In the test dataset')
for bp in df_test.body_part.unique():
    sp0=df_test[(df_test.body_part==bp) & (df_test.label==0)].shape[0]
    sp1=df_test[(df_test.body_part==bp) & (df_test.label==1)].shape[0]
    print(f'{bp} 0: {sp0} ({100*sp0/(sp0+sp1):.2f}%)')
    print(f'{bp} 1: {sp1} ({100*sp1/(sp0+sp1):.2f}%)')
    print(f'{bp} all: {sp0+sp1} ({100*(sp0+sp1)/(df_test.shape[0]):.2f}%)')

In [ ]:
print(f'train: {df_train.shape[0]} ({100*df_train.shape[0]/(df_train.shape[0]+df_test.shape[0]):.2f}%)')
print(f'test: {df_test.shape[0]} ({100*df_test.shape[0]/(df_train.shape[0]+df_test.shape[0]):.2f}%)')

In [ ]:
from skimage.io import imread
sub_df = dataset_all.groupby(['body_part', 'label']).apply(lambda x: x.sample(1)).reset_index(drop = True)
fig, (m_axs) = plt.subplots(2, sub_df.shape[0]//2, figsize = (12, 6))
for c_ax, (_, c_row) in zip(m_axs.flatten(), sub_df.iterrows()):
    c_ax.imshow(imread(c_row['img_path']), cmap = 'bone')
    c_ax.axis('off')
    c_ax.set_title('{body_part}:{label}'.format(**c_row))
fig.savefig('samples.png', dpi = 300)

In [ ]:
#csv files path for
path = 'MURA-v1.1'
train_image_paths_csv = "train_image_paths.csv"
train_images_paths = pd.read_csv(os.path.join(path,train_image_paths_csv),dtype=str,header=None)
train_images_paths.columns = ['image_path']
train_images_paths['label'] = train_images_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
train_images_paths['category']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[2])  
train_images_paths['patientId']  = train_images_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))
train_images_paths.head()

In [ ]:
path = 'MURA-v1.1'
valid_image_paths_csv = "valid_image_paths.csv"
valid_data_paths = pd.read_csv(os.path.join(path,valid_image_paths_csv),dtype=str,header=None)
valid_data_paths.columns = ['image_path']
valid_data_paths['label'] = valid_data_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
valid_data_paths['category']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[2])  
valid_data_paths['dir'] =  valid_data_paths['image_path'].apply(lambda x: x.split('/')[1])
valid_data_paths['patientId']  = valid_data_paths['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))
valid_data_paths.head()

In [ ]:
train_images_paths['label_index']= train_images_paths.label
train_images_paths.label_index.replace('positive', 1, inplace=True)
train_images_paths.label_index.replace('negative', 0, inplace=True)
train_images_paths.head(3)

In [ ]:
valid_data_paths['label_index']= valid_data_paths.label
valid_data_paths.label_index.replace('positive', 1, inplace=True)
valid_data_paths.label_index.replace('negative', 0, inplace=True)
valid_data_paths.head(3)

In [ ]:
im_size = 256
def random_rotation_flip(image,size = 256):
    if random.randint(0,1):
        image = cv2.flip(image,1) # 1-->horizontal flip 0-->Vertical flip -1-->Horizontal and vertical

    if random.randint(0,1):
            angle = random.randint(-30,30)
            M = cv2.getRotationMatrix2D((size/2,size/2),angle,1)
            #The third parameter: the size of the transformed image
            image = cv2.warpAffine(image,M,(size,size))
    return image
def image_loader(Path, size = 224): 
    Images = []
    for path in tqdm(Path):
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            image = random_rotation_flip(image,size)
            Images.append(image)
        except Exception as e:
            print(str(e))   
    Images = np.asarray(Images).astype('float32')
    mean = np.mean(Images)
    std = np.std(Images)
    Images = (Images - mean) / std
    
    return Images
X_train = image_loader(train_images_paths['image_path'][:50,],im_size)
y_train = train_images_paths['label']
Y_train = y_train.replace("positive",1)
Y_train = Y_train.replace("negative",0)

X_test = image_loader(valid_data_paths['image_path'][:50,],im_size)
y_test = valid_data_paths['label']
Y_test = y_test.replace("positive",1)
Y_test = Y_test.replace("negative",0)

train, valid = train_test_split(train_images_paths, test_size=0.2,random_state=seed)

In [ ]:
test = valid_data_paths.drop(['dir'], axis=1)

In [ ]:
image_generator_settings = dict(
                          rescale = 1. / 255,
                          #samplewise_center = True,
                          #samplewise_std_normalization = True
                          #rotation_range = 5, 
                         )
image_generator = ImageDataGenerator(**image_generator_settings)

In [ ]:
path = 'MURA-v1.1'
train_generator = image_generator.flow_from_dataframe(dataframe = train,directory = None,x_col = 'image_path',y_col = 'label_index',batch_size = 64,shuffle = True,class_mode = 'raw', target_size = (im_size, im_size),color_mode = 'rgb',interpolation='nearest',validate_filenames=False,seed=seed)
valid_generator = image_generator.flow_from_dataframe(dataframe = valid,directory = None,x_col = 'image_path',y_col = 'label_index',batch_size = 64,shuffle = True,class_mode = 'raw',target_size = (im_size, im_size),color_mode = 'rgb',interpolation='nearest',validate_filenames=True,seed=seed)
test_generator = image_generator.flow_from_dataframe(dataframe = test,directory = None,x_col = 'image_path',y_col = 'label_index',batch_size = 64,shuffle = False,class_mode = 'raw', target_size = (im_size, im_size),color_mode = 'rgb',interpolation='nearest', validate_filenames=True,seed=seed)
CLASSES = 2
input_shape = (im_size,im_size,3)

In [ ]:
early_stop = EarlyStopping(monitor="val_kappa",mode="max", patience=3,restore_best_weights=True)
epochs=10
model=cnn_model41max
model.compile(optimizer=Adam(), loss= BinaryCrossentropy(from_logits=False),metrics=[metrics()])
history=model.fit(train_generator,validation_data = valid_generator, epochs = epochs,callbacks=[early_stop])

In [ ]:
model.evaluate(test_generator)

In [ ]:
def resnet_builder(
        pooling="max", 
        shape=(256, 256, 3), 
        trainable_layers_after=None
    ):
    resNet = ResNet152V2(
        include_top=False,
        weights='imagenet',
        input_shape=shape,
        pooling=pooling
    )
    if trainable_layers_after:
        for layer in resNet.layers[:trainable_layers_after]:
            layer.trainable = False
    else:
        resNet.trainable = False
    prediction_layer = Dense(1, activation="sigmoid",
                                name="resnet_output_sigmoid")
    model = Sequential(
        layers=[
            resNet,
            prediction_layer
        ],
        name="resnet"
    )
    return model

In [ ]:
epochs = 50
early_stop = EarlyStopping(monitor="kappa",mode="min", patience=3, restore_best_weights=True)
resnet_model = resnet_builder(pooling='avg')
resnet_model.summary()
resnet_model.compile(optimizer=Adam(), loss= BinaryCrossentropy(from_logits=False),metrics=[metrics()])
hs = resnet_model.fit(train_generator,validation_data = valid_generator, epochs = epochs,callbacks=[early_stop])
print('Finished training.')
print('------------------')
resnet_model.summary()
filename = 'resnet1511.h5'
resnet_model.save(filename)

In [ ]:
resnet_model.evaluate(test_generator)

In [ ]:
epochs = 50
early_stop = EarlyStopping(monitor="val_kappa",mode="max", patience=3, restore_best_weights=True)
resnet_model = resnet_builder(pooling='max') #change between max and avg 
resnet_model.summary()
resnet_model.compile(optimizer=Adam(), loss= BinaryCrossentropy(from_logits=False),metrics=[metrics()])
hs = resnet_model.fit(train_generator,validation_data = valid_generator, epochs = epochs,callbacks=[early_stop])
print('Finished training.')
print('------------------')
resnet_model.summary()
filename = 'resnet1511.h5'
resnet_model.save(filename)

In [ ]:
resnet_model.evaluate(test_generator)

In [ ]:
resnet_model.evaluate(train_generator)

In [ ]:
def VGGNET16_builder(
        pooling="max", 
        shape=(256, 256, 3), 
        trainable_layers_after=None
    ):
    VGGNET16 = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=shape,
        pooling=pooling
    )
    if trainable_layers_after:
        for layer in VGGNET16.layers[:trainable_layers_after]:
            layer.trainable = False
    else:
        VGGNET16.trainable = False
    prediction_layer = Dense(1, activation="sigmoid",name="VGGNET_output_sigmoid")
    model = Sequential(
        layers=[
            VGGNET16,
            prediction_layer
        ],
        name="VGGNET16"
    )
    return model

In [ ]:
epochs = 10
early_stop = EarlyStopping(monitor="kappa", mode="min", patience=3, restore_best_weights=True)
VGGNET_model16 = VGGNET16_builder(pooling='max')
VGGNET_model16.summary
VGGNET_model16.compile(optimizer=Adam(),loss= BinaryCrossentropy(from_logits=False),metrics=[metrics()])
hs = VGGNET_model16.fit(train_generator,validation_data = valid_generator, epochs = epochs,callbacks=[early_stop])
print('Finished training.')
print('------------------')
VGGNET_model16.summary()
filename = 'vgg16.h5'
VGGNET_model16.save(filename)

In [ ]:
VGGNET_model16.evaluate(test_generator)

In [ ]:
def densenet_builder(
        pooling="avg",
        shape=(256, 256, 3),
        trainable_layers_after=None,
        mlp=[],
        mlp_dropout=0.25,
        nameNN="",
):
    denseNet = DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=shape,
        pooling=pooling
    )
    if trainable_layers_after:
        for layer in denseNet.layers[:trainable_layers_after]:
            layer.trainable = False
    else:
        denseNet.trainable = False
    output = denseNet.output
    for index, mlp_neurons in enumerate(mlp):
        output = Dense(mlp_neurons, activation="relu", name=f"m.{index}.{mlp_neurons}")(output)
        if mlp_dropout:
            output = Dropout(mlp_dropout, name=f"mdp.{index}.{mlp_neurons}")(output)
    output = Dense(1, activation="sigmoid", name="densenet_output_sigmoid")(output)
    model = Model(denseNet.input, output, name='densenet'+nameNN)
    return model

In [ ]:
epochs = 4
early_stop = EarlyStopping(monitor="val_kappa", mode="max", patience=2,restore_best_weights=True)
densenet_model = densenet_builder(pooling='avg')
densenet_model.compile(optimizer=Adam(), 
                  loss= BinaryCrossentropy(from_logits=False),
                  metrics=[metrics()])
# hs = densenet_model.fit(
#         train_generator,
#         validation_data = valid_generator, 
#         epochs = epochs,
#         callbacks=[early_stop]
#     )
print('Finished training.')
print('------------------')
densenet_model.evaluate(test_generator)
filename = 'densenet.h5'
densenet_model.save(filename)